# Demonstrate Triggers

In [ ]:
from DATA225utils import make_connection, dataframe_query

In [ ]:
conn = make_connection(config_file = 'pay.ini')
cursor = conn.cursor()

## Payment table

In [ ]:
cursor.execute('DROP TABLE IF EXISTS payment')

sql = ( """
        CREATE TABLE payment
        (
            id INT UNIQUE NOT NULL AUTO_INCREMENT,
            name VARCHAR(16) UNIQUE NOT NULL,
            rate DOUBLE NOT NULL,
            PRIMARY KEY (id)
        )
        """
    )

cursor.execute(sql);

## Log table

In [ ]:
cursor.execute('DROP TABLE IF EXISTS log')

sql = ( """
        CREATE TABLE log
        (
            id INT UNIQUE NOT NULL AUTO_INCREMENT,
            timestamp DATETIME NOT NULL,
            name VARCHAR(16) UNIQUE NOT NULL,
            rate DOUBLE NOT NULL,
            PRIMARY KEY (id)
        )
        """
)

cursor.execute(sql);

## Initial data

In [ ]:
sql = ( """
        INSERT INTO payment(name, rate)
        VALUES (%s, %s)
        """
      ) 

data = [ ('Bob',   27), 
         ('Frank', 25), 
         ('Mary',  30)
       ]

cursor.executemany(sql, data)  # NOTE: executemany()
conn.commit()

In [ ]:
_, df = dataframe_query(conn, 'SELECT * FROM payment')
df

## Trigger: Before

#### **Before** inserting a new employee, calculate the employee's rate to be $1 more than the current minimum rate.

In [ ]:
cursor.execute('DROP TRIGGER IF EXISTS insert_new_employee')

sql = ( """
        CREATE TRIGGER insert_new_employee
            BEFORE INSERT ON payment
            FOR EACH ROW
        BEGIN
            DECLARE min_rate DOUBLE;
            SELECT MIN(rate) FROM payment INTO min_rate;
            SET NEW.rate = min_rate + 1;
        END
        """
      )

cursor.execute(sql)

## Trigger: After

#### **After** insering a new employee, log the insertion.

In [ ]:
cursor.execute('DROP TRIGGER IF EXISTS log_new_employee')

sql = ( """
        CREATE TRIGGER log_new_employee
            AFTER INSERT ON payment
            FOR EACH ROW
        BEGIN
            INSERT INTO log(timestamp, name, rate)
                VALUES (NOW(), NEW.name, NEW.rate);
        END
        """
      )

cursor.execute(sql)

In [ ]:
_, df = dataframe_query(conn, 'SHOW TRIGGERS')
df

## Insert new employees

In [ ]:
sql = ( """
        INSERT INTO payment(name) 
        VALUES ('ron')
        """
      )

cursor.execute(sql)
conn.commit()

In [ ]:
_, df = dataframe_query(conn, 'SELECT * FROM payment')
df

In [ ]:
sql = ( """
        INSERT INTO payment(name) 
        VALUES ('sara')
        """
      )

cursor.execute(sql)
conn.commit()

In [ ]:
_, df = dataframe_query(conn, 'SELECT * FROM payment')
df

## What got logged

In [ ]:
_, df = dataframe_query(conn, 'SELECT * FROM log')
df

In [ ]:
cursor.close()
conn.close()